In [22]:
from librerias import *

In [23]:
ruta_archivo = 'Data/btc_limp.csv' 

btc = pd.read_csv(ruta_archivo)

btc['Date'] = pd.to_datetime(btc['Date'])

btc_week = btc.groupby(pd.Grouper(key='Date', freq='W')).last()

btc.head ()

,Date,Open,Change %
0,2024-03-18,68389,-1.17
1,2024-03-17,65314,4.71
2,2024-03-16,69456,-5.97
3,2024-03-15,71387,-2.69
4,2024-03-14,73066,-2.30


In [24]:
btc_week.index = pd.to_datetime(btc_week.index)

# Cortar el DataFrame desde la fecha especificada en adelante
btc_week = btc_week.loc['2014-01-05':]

# Mostrar el DataFrame resultante
print(btc_week)

             Open  Change %
Date                       
2014-01-05    924      9.74
2014-01-12   1005     -6.52
2014-01-19    905      5.42
2014-01-26    961      4.78
2014-02-02    940      1.37
...           ...       ...
2024-02-25  51572      0.29
2024-03-03  61955      1.84
2024-03-10  68360      1.57
2024-03-17  65314      4.71
2024-03-24  68389     -1.17

[534 rows x 2 columns]


In [25]:
# Estrategia 1. Inversión y hold

# Convertir la columna 'Open' a tipo numérico
btc_week[' Open'] = pd.to_numeric(btc_week['Open'], errors='coerce')

# obtener el precio de bitcoin primer y ultimo día

precio_primer_dia = btc_week['Open'].iloc[0]
precio_ultimo_dia = btc_week['Open'].iloc[-1]

# Calcular cuantos bitcoin podrias comprar el primer día

bitcoins_comprados_primer_dia = 1 / precio_primer_dia

# Calcular el valor actual de la inversion para cada fecha

btc_week['valor_actual'] = bitcoins_comprados_primer_dia * btc_week['Open']

btc_week['valor_actual'] = btc_week['valor_actual'].round(2)

In [26]:
btc_week

,Open,Change %,Open,valor_actual
Date,,,,
2014-01-05,924,9.74,924,1.00
2014-01-12,1005,-6.52,1005,1.09
2014-01-19,905,5.42,905,0.98
2014-01-26,961,4.78,961,1.04
2014-02-02,940,1.37,940,1.02
...,...,...,...,...
2024-02-25,51572,0.29,51572,55.81
2024-03-03,61955,1.84,61955,67.05
2024-03-10,68360,1.57,68360,73.98


In [27]:
import plotly.graph_objects as go

# Crear la figura
fig = go.Figure()

fig.add_trace(go.Scatter(y=btc_week['valor_actual'], x=btc_week.index,
                         mode='lines',
                         name='Valor actual'))
# Actualizar el diseño del gráfico
fig.update_layout(title='Valor actual de Bitcoin en función de la fecha',
                  xaxis_title='Valor actual',
                  yaxis_title='Fecha')

# Mostrar el gráfico
fig.show()


In [28]:
# Seleccionar los 15 números más altos de la columna 'Change %'
cambio_alto = btc_week['Change %'].nlargest(15)

# Seleccionar los 15 números más bajos de la columna 'Change %'
cambio_min = btc_week['Change %'].nsmallest(15) 

cambio_alto

Date
2014-02-23    21.35
2016-06-12    12.67
2019-05-19    12.32
2017-12-31    10.52
2021-06-13    10.02
2014-01-05     9.74
2014-05-25     9.60
2018-03-11     8.76
2022-06-19     8.38
2021-06-27     7.55
2019-05-26     7.51
2023-03-12     7.46
2017-10-29     7.35
2017-05-28     6.66
2017-11-26     6.44
Name: Change %, dtype: float64

In [29]:
# Crear una copia de btc_week
btc_sin_top = btc_week.copy()

# Obtener los índices de las 15 filas con los valores más altos en la columna 'Change %'
indices_altos = btc_sin_top['Change %'].nlargest(15).index

# Reemplazar los valores en esos índices con ceros
btc_sin_top.loc[indices_altos, 'Change %'] = 0
# Crear la figura

indices_altos




DatetimeIndex(['2014-02-23', '2016-06-12', '2019-05-19', '2017-12-31',
               '2021-06-13', '2014-01-05', '2014-05-25', '2018-03-11',
               '2022-06-19', '2021-06-27', '2019-05-26', '2023-03-12',
               '2017-10-29', '2017-05-28', '2017-11-26'],
              dtype='datetime64[ns]', name='Date', freq=None)

In [30]:

def calcular_valor_actual(df):
    # Convertir la columna 'Open' a tipo numérico
    df['Open'] = pd.to_numeric(df['Open'], errors='coerce')

    # obtener el precio de bitcoin primer y ultimo día
    precio_primer_dia = df['Open'].iloc[0]
    precio_ultimo_dia = df['Open'].iloc[-1]

    # Calcular cuantos bitcoin podrias comprar el primer día
    bitcoins_comprados_primer_dia = 1 / precio_primer_dia

    # Calcular el valor actual de la inversion para cada fecha
    df['valor_actual'] = bitcoins_comprados_primer_dia * df['Open']
    df['valor_actual'] = df['valor_actual'].round(2)
    
    return df

# Llama a la función con tu DataFrame btc_week
btc_sin_top = calcular_valor_actual(btc_sin_top)
btc_sin_top = calcular_valor_actual(btc_sin_top)


In [31]:

fig = go.Figure()

fig.add_trace(go.Scatter(y=btc_week['valor_actual'], x=btc_week.index,
                         mode='lines',
                         name='Valor actual'))

fig.add_trace(go.Scatter(y=btc_sin_top['valor_actual'], x=btc_sin_top.index,
                         mode='lines',
                         name='Valor actual (sin top)',
                         line=dict(color='red')))
# Actualizar el diseño del gráfico

fig.update_layout(title='Valor actual de Bitcoin en función de la fecha',
                  xaxis_title='Valor actual',
                  yaxis_title='Fecha')

# Mostrar el gráfico
fig.show()

In [32]:
print(btc_sin_top['valor_actual'].head())
print(btc_sin_top['valor_actual'].tail())


Date
2014-01-05    1.00
2014-01-12    1.09
2014-01-19    0.98
2014-01-26    1.04
2014-02-02    1.02
Freq: W-SUN, Name: valor_actual, dtype: float64
Date
2024-02-25    55.81
2024-03-03    67.05
2024-03-10    73.98
2024-03-17    70.69
2024-03-24    74.01
Freq: W-SUN, Name: valor_actual, dtype: float64


In [33]:
print(btc_week['valor_actual'].head())
print(btc_week['valor_actual'].tail())

Date
2014-01-05    1.00
2014-01-12    1.09
2014-01-19    0.98
2014-01-26    1.04
2014-02-02    1.02
Freq: W-SUN, Name: valor_actual, dtype: float64
Date
2024-02-25    55.81
2024-03-03    67.05
2024-03-10    73.98
2024-03-17    70.69
2024-03-24    74.01
Freq: W-SUN, Name: valor_actual, dtype: float64


In [34]:

def calcular_valor_actual(df):
    # Convertir la columna 'Open' a tipo numérico
    df['Open'] = pd.to_numeric(df['Open'], errors='coerce')

    # obtener el precio de bitcoin primer y ultimo día
    precio_primer_dia = df['Open'].iloc[0]
    precio_ultimo_dia = df['Open'].iloc[-1]

    # Calcular cuantos bitcoin podrias comprar el primer día
    bitcoins_comprados_primer_dia = 1 / precio_primer_dia

    # Calcular el valor actual de la inversion para cada fecha
    df['valor_actual'] = bitcoins_comprados_primer_dia * df['Open']
    df['valor_actual'] = df['valor_actual'].round(2)
    
    return df

# Llama a la función con tu DataFrame btc_week
btc_sin_top = calcular_valor_actual(btc_sin_top)
btc_week = calcular_valor_actual(btc_week)

In [35]:
print(btc_week['valor_actual'].head())
print(btc_week['valor_actual'].tail())

Date
2014-01-05    1.00
2014-01-12    1.09
2014-01-19    0.98
2014-01-26    1.04
2014-02-02    1.02
Freq: W-SUN, Name: valor_actual, dtype: float64
Date
2024-02-25    55.81
2024-03-03    67.05
2024-03-10    73.98
2024-03-17    70.69
2024-03-24    74.01
Freq: W-SUN, Name: valor_actual, dtype: float64


In [36]:
# Crear una copia de btc_week
btc_sin_top = btc_week.copy()

# Obtener los índices de las 15 filas con los valores más altos en la columna 'Change %'
indices_altos = btc_sin_top['Change %'].nlargest(15).index

# Eliminar las filas con esos índices
btc_sin_top = btc_sin_top.drop(indices_altos)

# Ahora btc_sin_top contiene todas las filas excepto las 15 con los valores más altos en 'Change %'

In [37]:
print(btc_sin_top['valor_actual'].head())
print(btc_sin_top['valor_actual'].tail())


Date
2014-01-12    1.09
2014-01-19    0.98
2014-01-26    1.04
2014-02-02    1.02
2014-02-09    0.70
Name: valor_actual, dtype: float64
Date
2024-02-25    55.81
2024-03-03    67.05
2024-03-10    73.98
2024-03-17    70.69
2024-03-24    74.01
Name: valor_actual, dtype: float64


In [38]:
ruta_archivo = 'Data/btc_limp.csv' 

btc = pd.read_csv(ruta_archivo)

btc['Date'] = pd.to_datetime(btc['Date'])

btc_week = btc.groupby(pd.Grouper(key='Date', freq='W')).last()

btc.head ()

,Date,Open,Change %
0,2024-03-18,68389,-1.17
1,2024-03-17,65314,4.71
2,2024-03-16,69456,-5.97
3,2024-03-15,71387,-2.69
4,2024-03-14,73066,-2.30


In [39]:
btc_ko = btc_week.copy ()

In [40]:
cambio = btc_ko ['Open']

In [41]:
acumul = (1+cambio).cumprod ()
acumul.fillna (1,inplace=True)

print (acumul)


Date
2010-07-25    1
2010-08-01    1
2010-08-08    1
2010-08-15    1
2010-08-22    1
             ..
2024-02-25    0
2024-03-03    0
2024-03-10    0
2024-03-17    0
2024-03-24    0
Freq: W-SUN, Name: Open, Length: 714, dtype: int64
